In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [2]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [3]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex2"
today

'day22'

In [4]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [5]:
secrets = [int(line) for line in get_lines()]
len(secrets)

2420

In [6]:
def mix(val: int, secret: int) -> int:
    return val ^ secret

def prune(secret: int) -> int:
    return secret % 16777216

def next_secret(secret: int) -> int:
    tmp = secret * 64
    secret = mix(tmp, secret)
    secret = prune(secret)

    tmp = secret // 32
    secret = mix(tmp, secret)
    secret = prune(secret)

    tmp = secret * 2048
    secret = mix(tmp, secret)
    secret = prune(secret)

    return secret

In [7]:
secret = 123
for _ in range(10):
    secret = next_secret(secret)
    print(secret)

15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254


In [8]:
s = 0
for secret in secrets:
    for _ in range(2000):
        secret = next_secret(secret)
    s += secret
s

19877757850

In [9]:
sequence_to_bananas = defaultdict(int)
for secret in secrets:
    seen = set()
    dq = deque()
    prev = secret % 10
    for _ in range(3):
        secret = next_secret(secret)
        curr = secret % 10
        dq.append(curr - prev)
        prev = curr
    for _ in range(2000 - 3):
        secret = next_secret(secret)
        curr = secret % 10
        dq.append(curr - prev)
        seq = tuple(dq)
        if seq not in seen:
            seen.add(seq)
            sequence_to_bananas[seq] += curr
        dq.popleft()
        prev = curr

In [10]:
max(sequence_to_bananas.values())

2399